In [1]:
!pip install kaggle


In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mihir3vedi","key":"760b971d3451795301ff7f9247e1a44e"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [4]:
!kaggle datasets download -d abdelghaniaaba/wildfire-prediction-dataset

100% 1.44G/1.45G [00:22<00:00, 60.1MB/s]
100% 1.45G/1.45G [00:22<00:00, 67.8MB/s]


In [ ]:
!unzip wildfire-prediction-dataset.zip


In [6]:
!pip install -q transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.4 MB/s eta 0:00:00


In [7]:
from transformers import ViTImageProcessor
from PIL import Image
import os

# Define the path to the "train" folder containing subfolders
train_folder = "/content/train"

# Initialize the image processor
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

# List all subdirectories in the "train" folder
subfolders = [f.path for f in os.scandir(train_folder) if f.is_dir()]

# Process images in each subfolder
for subfolder in subfolders:
    # List all image files in the current subfolder
    image_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for image_path in image_files:
        try:
            # Load the image
            image = Image.open(image_path)

            # Process the image using the ViTImageProcessor
            inputs = processor(images=image, return_tensors="pt")
            pixel_values = inputs.pixel_values

            # Now you can use pixel_values for further processing with your model
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
            # Delete the corrupted image
            os.remove(image_path)
            print(f"{image_path} deleted due to corruption")


Error processing /content/train/nowildfire/-114.152378,51.027198.jpg: image file is truncated (16 bytes not processed)
/content/train/nowildfire/-114.152378,51.027198.jpg deleted due to corruption


In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),           # Convert to a tensor
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize pixel values
])


In [18]:
train_data = datasets.ImageFolder('/content/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


29084464/29084464 [==============================] - 0s 0us/step


In [20]:
train_path ='/content/train'
validation_path = '/content/valid'
test_path = '/content/test'



image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


Found 30249 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [21]:
epochs = 10

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/densenet_model.h5', save_best_only=True)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


Epoch 1/10
946/946 [==============================] - 488s 498ms/step - loss: 0.2110 - accuracy: 0.9197 - val_loss: 0.1469 - val_accuracy: 0.9475
Epoch 2/10
946/946 [==============================] - 467s 493ms/step - loss: 0.1607 - accuracy: 0.9397 - val_loss: 0.1470 - val_accuracy: 0.9465
Epoch 3/10
946/946 [==============================] - 458s 484ms/step - loss: 0.1470 - accuracy: 0.9440 - val_loss: 0.1600 - val_accuracy: 0.9410
Epoch 4/10
946/946 [==============================] - 461s 487ms/step - loss: 0.1410 - accuracy: 0.9464 - val_loss: 0.1344 - val_accuracy: 0.9506
Epoch 5/10
946/946 [==============================] - 457s 483ms/step - loss: 0.1364 - accuracy: 0.9472 - val_loss: 0.1156 - val_accuracy: 0.9563
Epoch 6/10
946/946 [==============================] - 464s 490ms/step - loss: 0.1327 - accuracy: 0.9502 - val_loss: 0.1169 - val_accuracy: 0.9560
Epoch 7/10
946/946 [==============================] - 461s 488ms/step - loss: 0.1272 - accuracy: 0.9520 - val_loss: 0.1076 -

KeyboardInterrupt: ignored